In [2]:
import numpy as np
import pandas as pd
import networkx as nx

In [47]:
#1 this is my model

#lets first create the state for hmm 
states =["rain","no_rain"]
states_ = pd.DataFrame(columns=states,index=states)

states_.loc[states[0]] = [0.7,0.3]
states_.loc[states[1]] = [0.3,0.7]



#initial Probabilities 
pi = [0.5,0.5]

observation = ["umbrella","noUmberalla"]

observation_ = pd.DataFrame(columns=observation,index=states)
observation_.loc[:,observation[0]] = [0.9,0.1]
observation_.loc[:,observation[1]] = [0.2,0.8]

observation_

,umbrella,noUmberalla
rain,0.9,0.2
no_rain,0.1,0.8


In [4]:
observation_['umbrella']['rain']

0.9

In [5]:
from pprint import pprint

def create_graph(Q):
    edg={}
    for col in Q.columns:
        for idx in Q.index:
            edg[(idx,col)] = Q[idx][col]
        
    return edg
    

edges_weight = create_graph(states_)
pprint(create_graph(states_))


{('no_rain', 'no_rain'): 0.7,
 ('no_rain', 'rain'): 0.3,
 ('rain', 'no_rain'): 0.3,
 ('rain', 'rain'): 0.7}


In [6]:
graph = nx.MultiDiGraph()

graph.add_nodes_from(states)

for k,v in edges_weight.items():
    origin_ , destination_ = k[0],v
    graph.add_edge(origin_,destination_,weight =v,label=v)
    
pprint(graph.edges(data=True))

pos = nx.drawing.nx_pydot.graphviz_layout(graph, prog='dot')
nx.draw_networkx(graph, pos)

edge_labels = {(n1,n2):d['label'] for n1,n2,d in graph.edges(data=True)}
nx.draw_networkx_edge_labels(graph, pos, edge_labels=edge_labels)
nx.drawing.nx_pydot.write_dot(graph, 'pet_dog_markov.dot')

OutMultiEdgeDataView([('rain', 0.7, {'weight': 0.7, 'label': 0.7}), ('rain', 0.3, {'weight': 0.3, 'label': 0.3}), ('no_rain', 0.3, {'weight': 0.3, 'label': 0.3}), ('no_rain', 0.7, {'weight': 0.7, 'label': 0.7})])


In [7]:
def vertibi(mood_list):
    memory = []
    if mood_list[0] == 'umbrella':
        #p(rain) * p(rain|umbrella)
        #p(not_rain) *p(not_rain|umbrella)
     
        
        rain = pi[0]*observation_["umbrella"]["rain"]
        no_rain = pi[1]*observation_["umbrella"]["no_rain"]
    
        memory.append((rain,no_rain))
    else:
        rain = pi[0]*observation_["noUmberalla"]["rain"]
        no_rain = pi[1]*observation_["noUmberalla"]["no_rain"]
    
        memory.append((rain,no_rain))
        
    
    for m_d in range(1,len(mood_list)):
        rain,no_rain = memory[-1]
        
        if mood_list[m_d] == 'umbrella':
            #for rain
            rain = max(rain*states_["rain"]["rain"]*observation_["umbrella"]["rain"],no_rain*states_["no_rain"]["rain"]\
            *observation_["umbrella"]["rain"])
            
            
            
            #for no rain
            no_rain = max(rain*states_["rain"]["no_rain"]*observatioumbrellan_["umbrella"]["no_rain"],no_rain*states_["no_rain"]["no_rain"]\
            *observation_["umbrella"]["no_rain"])
            
            memory.append((rain,no_rain))
            
        else:
            
           #for rain
            rain = max(rain*states_["rain"]["rain"]*observation_["noUmberalla"]["rain"],no_rain*states_["no_rain"]["rain"]\
            *observation_["noUmberalla"]["rain"])
            
            
            
            #for no rain
            no_rain = max(rain*states_["rain"]["no_rain"]*observation_["noUmberalla"]["no_rain"],no_rain*states_["no_rain"]["no_rain"]\
            *observation_["noUmberalla"]["no_rain"])
            
            memory.append((rain,no_rain))

    return memory    
            
            
        
        
        
    

    

In [8]:
def directSampling():
    uniform_distribution = np.random.uniform(0,1,20)
   
    total_sequence =  15
    count =0
    myObservation =[]
    myState       =[]
    track_uniform_distribution = {}
    for seq in range(total_sequence):
        seq_sample = []
        obs_sample = []
       
        for val  in uniform_distribution:
          
                if val > 0 and val <= pi[0]:
                    
                    rain = "rain"
                    #probability of rain | rain and no_rain|rain
                    
                    rain_given_rain = states_['rain']['rain']
                    no_rain_given_rain = states_['no_rain']['rain']
                    
                    
                    if val >  0  and val <= rain_given_rain:
                        seq_sample.append('rain')
                        #find the observation for this umbrella|rain , not_umbrella|rain
                        umbrella_given_rain = observation_['umbrella']['rain']
                        not_umbrella_given_rain = observation_['noUmberalla']['rain']
                        
                        if val >0 and val <= umbrella_given_rain:
                            obs_sample.append('umbrella')
                        else:
                            obs_sample.append('noUmberalla')
                            
                    elif val > rain_given_rain and val <= no_rain_given_rain+rain_given_rain:
                        seq_sample.append('no_rain')
                        
                         #find the observation for this umbrella|rain , not_umbrella|rain
                        umbrella_given_rain = observation_['umbrella']['no_rain']
                        not_umbrella_given_rain = observation_['noUmberalla']['no_rain']
                        
                        if val >0 and val <= umbrella_given_rain:
                            obs_sample.append('umbrella')
                        else:
                            obs_sample.append('noUmberalla')
                        
                    
                    
                else:
                    
                    rain = "no_rain"
                    #probability of rain | rain and no_rain|rain
                    
                    rain_given_rain = states_['rain']['no_rain']
                    no_rain_given_rain = states_['no_rain']['no_rain']
                    
                    
                    if val >  0  and val <= rain_given_rain:
                        seq_sample.append('rain')
                        #find the observation for this umbrella|rain , not_umbrella|rain
                        umbrella_given_rain = observation_['umbrella']['rain']
                        not_umbrella_given_rain = observation_['noUmberalla']['rain']
                        
                        if val >0 and val <= umbrella_given_rain:
                            obs_sample.append('umbrella')
                        else:
                            obs_sample.append('noUmberalla')
                            
                    elif val > rain_given_rain and val <= no_rain_given_rain+rain_given_rain:
                        seq_sample.append('no_rain')
                        
                         #find the observation for this umbrella|rain , not_umbrella|rain
                        umbrella_given_rain = observation_['umbrella']['no_rain']
                        not_umbrella_given_rain = observation_['noUmberalla']['no_rain']
                        
                        if val >0 and val <= umbrella_given_rain:
                            obs_sample.append('umbrella')
                        else:
                            obs_sample.append('noUmberalla')
        
        myObservation.append(obs_sample)
        myState.append(seq_sample)
        
        count+=1
        
    return myObservation,myState
        

In [139]:

def normalize(matrix,cons):
    matrix[0][0] = matrix[0][0]/cons
    matrix[1][0] = matrix[1][0]/cons
    return matrix
        
        
    


def forward_algorithm(observation , state):
    #lets calculate the initial probabilities
    prior  = np.array(pi).reshape(-1,1)
    memory = []
    prior  = np.asmatrix(prior)
    transition_prob  =  np.asmatrix(states_.values)
    for ob in range(len(observation)):
        #find the probability of the state given observation
        
        obs_model =  np.zeros((2,2))
            
        prob_observation = observation_.loc[state[ob]].values
            
        obs_model[0][0] = prob_observation[0]
        obs_model[1][1] = prob_observation[1]
            
        obs_model  = np.asmatrix(obs_model)
            
        matrix_result  = obs_model*transition_prob
        new_prior  =  matrix_result*prior
            
        cons  = np.sum(new_prior)
            
        new_prior = normalize(new_prior,cons)
            
        memory.append(new_prior)
            
        prior = new_prior
            
            
    return memory 
            

In [229]:
def backward(observation,state):
    prior = np.array([1,1])
    prior =  prior.reshape(-1,1)
    prior_ = np.asmatrix(prior)
    
    
    backward_memory  = []
    transition_prob  =  np.asmatrix(states_.values)
    
    for ob in range(len(observation)):
        
        obs_model =  np.zeros((2,2))

        prob_observation = observation_.loc[state[ob]].values

        obs_model[0][0] = prob_observation[0]
        obs_model[1][1] = prob_observation[1]

        obs_model  = np.asmatrix(obs_model)

        matrix_result  = transition_prob*obs_model
        new_prior_  =  matrix_result*prior_

        cons  = np.sum(new_prior_)

        new_prior_ = normalize(new_prior_,cons)

        backward_memory.append(new_prior_)

        prior_ = new_prior_
        
    
    return backward_memory
            
            

def backwardMatmulwithforward(backward_memory,memory):
    
    result=[]
    prior = np.array(pi)
    prior = prior.reshape(-1,1)
    prior = np.asmatrix(prior)
    
    
    result.append(np.multiply(prior,backward_memory[-1]))
    
    
    count = -2
    
    for forward_ in memory:
        result.append(np.multiply(prior,backward_memory[count]))
        count = count -1
        if np.abs(count) > 20:
            break
        
    return result
    
        
        
def checking(result):
    state_observation = []
    observation_future =[]
    for i in result:
        coun = np.argmax(i)
        if coun == 0:
            state_observation.append("rain")
            observation_future.append("umbrella")
        else:
            state_observation.append("no_rain")
            observation_future.append("noUmbrella")
            
    return (state_observation,observation_future)
 
            
              
    

    
    
    
    

In [239]:
observation,state  = directSampling()

In [165]:
forward_memory  = forward_algorithm(observation[0],state[0])

In [166]:
backward_memory  = backward(observation[0],state[0])

In [218]:
future_observation = backwardMatmulwithforward(backward_memory,forward_memory)

In [231]:
state_observation_future,observation_future = checking(future_observation)

In [241]:
count =0
correctly_classified = 0
missclassified = 0
k_state = state[0]
for state__ in state_observation_future:
    if state__ == k_state[count]:
        correctly_classified+=1
    else:
        missclassified+=1
    count+=1
    
    

In [244]:
print("Correctly classified ",correctly_classified)
print("missclassified ",missclassified)

Correctly classified  11
missclassified  9
